In [ ]:
!pip -q install  openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


# Fine Tuning OpenAI GPT-3.5-turbo

A lot taken from:
https://github.com/openai/openai-cookbook

In [ ]:
import openai
import os

openai.api_key = ""

## Prepare your data

In [ ]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [ ]:
system_message = """
انت مساعد شخصي لتلاميذ باكالوريا الجزائر
عليك ان تجيب عللا اسئلتهم في حدود اليانات التي تم تدريبك عليها
حاول ان تحعل رسالتك مطولة نوعا ما

"""

dataset = [

{
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الرياضيات"},
            {"role": "assistant", "content": "سلاسل الأستاذ بلقاسم عبد الرزاق والأستاذ مصطفاي والأستاذ علالو"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الفيزياء"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين علوم الطبيعة والحياة"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع وسلاسل بوالريش"}
        ]
    },

   ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الانجليزية"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الفرنسية"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين التاريخ والجغرافيا"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الادب العربي"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة فهي غنية بالأفكار وهده الأفكار تتكرركثيرا لدا احتمال وجودها في السنة الحالية كبير"}
        ]
    },
  ##################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الاسبانية"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الألمانية"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الشريعة الإسلامية"},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة"}
        ]
    },

  ##########################################
 {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد سلسلة لتمارين الانجليزية "},
            {"role": "assistant", "content": "تجدها في صفحة الأستاذ عقبة بن نافع اضافة الى البكالوريات السابقة "}
        ]
    },

 ########################################################################################################################################################################################################################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد مراجع للمحاسبة "},
            {"role": "assistant", "content": "الأستاذ عبد الفتاح, الأستاذ رابح بقباقي"}
        ]
    },


  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد مراجع للإسبانية "},
            {"role": "assistant", "content": "Senor Rihi, hocine hm"}
        ]
    },

  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اريد مراجع للفرنسية "},
            {"role": "assistant", "content": "الأستاذ منصوري، الأستاذ منير شبوة"}
        ]
    },

  ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للعلوم "},
        {"role": "assistant", "content": "Mustapha bdd, ربيع ياسين, كتفي شريف زينة, شاوش"}
    ]},



  ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع شعبة رياضيات "},
        {"role": "assistant", "content": "الأستاذ نور الدين، الأستاذ جوفر، الأستاذ فرحاتي جيلالي، الأستاذ عبد الباسط، الأستاذ يونس بوسيف"}
    ]},


    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للفيزياء "},
        {"role": "assistant", "content": "الأستاذ ترير، الأستاذ زيدون، الأستاذ مولاي عمار، الأستاذ طيايبة عبد الرزاق"}
    ]},

    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للإجتماعيات "},
        {"role": "assistant", "content": "الأستاذ بورنان، كتاب الأستاذ بورنان، الأستاذ قنشوبة، كتاب الأستاذ محمودي عادل"}
    ]},



    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للتاريخ و جغرافيا "},
        {"role": "assistant", "content": "الأستاذ بورنان، كتاب الأستاذ بورنان، الأستاذ قنشوبة، كتاب الأستاذ محمودي عادل "}
    ]},



    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للإنجليزية "},
        {"role": "assistant", "content": "الأستاذ ناصري، الأستاذ منصوري"}
    ]},

    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع لمادة الفلسفة مراجع للفلسفة "},
        {"role": "assistant", "content": "الأستاذ خليل سعيداني، قناة هواري فيلو، الأستاذ عادل مقرود، كتاب هبة جبابرية"}
    ]},


    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للعربية "},
        {"role": "assistant", "content": "الأستاذ بوبكر، الأستاذ حيقون، الأستاذ خالد حماش"}
    ]},


    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للشريعة "},
        {"role": "assistant", "content": "كتاب الأستاذة بوسعادي، الأستاذ شمس الدين حماش"}
    ]},

  {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للشريعة "},
        {"role": "assistant", "content": "كتاب الأستاذة بوسعادي، الأستاذ شمس الدين حماش"}
    ]},
    ##########################################
    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للألمانية "},
        {"role": "assistant", "content": "الأستاذ حريزي، الأستاذ باسم داموس"}
    ]},

    {"messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "اريد مراجع للألمانية "},
        {"role": "assistant", "content": "الأستاذ حريزي، الأستاذ باسم داموس"}
    ]},

 ########################################################################################################################################################################################################################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ماهي أهم الكتب الخارجية لمادة الرياضيات"},
            {"role": "assistant", "content": "سلسلة الهدف \nكتب الاستاذ نورالدين \nكتب الاستاذ جوفر \nالمغني في الرياضيات \nالمراجعة النهائية في الرياضيات"}
        ]
    },

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ماهي أهم الكتب الخارجية لمادة الرياضيات"},
            {"role": "assistant", "content": "سلسلة الهدف \nكتب الاستاذ نورالدين \nكتب الاستاذ جوفر \nالمغني في الرياضيات \nالمراجعة النهائية في الرياضيات"}
        ]
    },
 ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العلوم الاسلامية/الشريعة"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية \nكتاب السلسلة الخضراء"}
        ]
    },

  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العلوم الاسلامية/الشريعة"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية \nكتاب السلسلة الخضراء"}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة التاريخ و الجغرافيا"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية في الاجتماعيات الأستاذ بورنان عمار اضافة الى كتاب محمودي عادل"}
        ]
    },

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة التاريخ و الجغرافيا"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية في الاجتماعيات الأستاذ بورنان عمار اضافة الى كتاب محمودي عادل"}
        ]
    },
 ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة علوم الطبيعة و الحياة"},
            {"role": "assistant", "content": "كتاب المراجعة النهائية في علوم الطبيعة إضافة إلى كتاب السلسلة الفضية وهو كتاب متوفر لشعبتي الرياضيات والعلوم التجريبية"}
        ]
    },

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة علوم الطبيعة و الحياة"},
            {"role": "assistant", "content": "كتاب المراجعة النهائية في علوم الطبيعة إضافة إلى كتاب السلسلة الفضية وهو كتاب متوفر لشعبتي الرياضيات والعلوم التجريبية"}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية للغات الأجنبية (الإنجليزية الفرنسية الألمانية الاسبانية )"},
            {"role": "assistant", "content": "تعتبر السلسلة الفضية من أفضل الخيارات"}
        ]
    },

      {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية للغات الأجنبية (الإنجليزية الفرنسية الألمانية الاسبانية )"},
            {"role": "assistant", "content": "تعتبر السلسلة الفضية من أفضل الخيارات"}
        ]
    },
 ##########################################
  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العربية / الادب العربي / اللغة العربية"},
            {"role": "assistant", "content": "المراجعة النهائية في اللغة العربية وآدابها \n كتب الاستاذ حيقون \n كتب الأستاذ حماش "}
        ]
    },

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العربية / الادب العربي / اللغة العربية"},
            {"role": "assistant", "content": "المراجعة النهائية في اللغة العربية وآدابها \n كتب الاستاذ حيقون \n كتب الأستاذ حماش "}
        ]
    },
 ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة التاريخ و الجغرافيا"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية في الاجتماعيات الأستاذ بورنان عمار اضافة الى كتاب محمودي عادل "}
        ]
    },

  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة التاريخ و الجغرافيا"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية في الاجتماعيات الأستاذ بورنان عمار اضافة الى كتاب محمودي عادل "}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة علوم الطبيعة و الحياة"},
            {"role": "assistant", "content": "كتاب المراجعة النهائية في علوم الطبيعة إضافة إلى كتاب السلسلة الفضية وهو كتاب متوفر لشعبتي الرياضيات والعلوم التجريبية"}
        ]
    },


    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة علوم الطبيعة و الحياة"},
            {"role": "assistant", "content": "كتاب المراجعة النهائية في علوم الطبيعة إضافة إلى كتاب السلسلة الفضية وهو كتاب متوفر لشعبتي الرياضيات والعلوم التجريبية"}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية للغات الأجنبية (الإنجليزية الفرنسية الألمانية الاسبانية )"},
            {"role": "assistant", "content": "تعتبر السلسلة الفضية من أفضل الخيارات "}
        ]
    },

  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية للغات الأجنبية (الإنجليزية الفرنسية الألمانية الاسبانية )"},
            {"role": "assistant", "content": "تعتبر السلسلة الفضية من أفضل الخيارات "}
        ]
    },

##########################################

    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة الفيزياء/ العلوم الفيزيائية"},
            {"role": "assistant", "content": "المغني في الفيزياء \n المراجعة النهائية في الفيزياء \n كتاب تأشيرة النجاح \n "}
        ]
    },

  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة الفيزياء/ العلوم الفيزيائية"},
            {"role": "assistant", "content": "المغني في الفيزياء \n المراجعة النهائية في الفيزياء \n كتاب تأشيرة النجاح \n "}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العلوم الاسلامية/الشريعة"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية \n كتاب السلسلة الخضراء"}
        ]
    },

     {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر أهم الكتب الخارجية لمادة العلوم الاسلامية/الشريعة"},
            {"role": "assistant", "content": "كتاب السلسلة الأرجوانية \n كتاب السلسلة الخضراء"}
        ]
    },


########################################################################################################################################################################################################################################

    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "كيف أختار التخصص الجامعي"},
            {"role": "assistant", "content": "أولا: اختر ما تريد و ما تحبه أنت إن كان معدلك مقبول في التخصص، لأنك أنت من ستدرسه لهذا اختر ما يناسبك و يناسب قدراتك.\n ثانيا: لا تعتمدوا على تجارب غيركم بصفة كلية في اختيار التخصص بل خدوها كمعيار فقط، لأن لكل شخص أسلوبه في الدراسة و ربما أسلوبك لا يلائمه هذا التخصص و يلائمه تخصص آخر.\nثالثا: إذا لم تقرر بعد، تمهل ابحث عن كل تخصص و فروعه و اختر بدقة، و لا تنظر إلى المدى القريب بل إلى ثلاثة سنوات أو أكثر في هذا التخصص و ربما إلى ما بعده (منصب العمل لمن يريد العمل بميدان دراسته)\nرابعا: كم من شخص وضع التخصص الذي يريده أولا ثم أهمل باقي الرغبات و لسوء حظه كان من نصيبه إحدى التخصصات التي لم يلقي لها بالا و وضعها في الترتيب الثاني أو الثالث أو حتى الرابع، و لهذا احسنوا الترتيب، فالتخصصات التي ستجدونها ليست كلها تدرس بمعدلكم."}
        ]
    },

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "كيف أختار التخصص الجامعي"},
            {"role": "assistant", "content": "أولا: اختر ما تريد و ما تحبه أنت إن كان معدلك مقبول في التخصص، لأنك أنت من ستدرسه لهذا اختر ما يناسبك و يناسب قدراتك.\n ثانيا: لا تعتمدوا على تجارب غيركم بصفة كلية في اختيار التخصص بل خدوها كمعيار فقط، لأن لكل شخص أسلوبه في الدراسة و ربما أسلوبك لا يلائمه هذا التخصص و يلائمه تخصص آخر.\nثالثا: إذا لم تقرر بعد، تمهل ابحث عن كل تخصص و فروعه و اختر بدقة، و لا تنظر إلى المدى القريب بل إلى ثلاثة سنوات أو أكثر في هذا التخصص و ربما إلى ما بعده (منصب العمل لمن يريد العمل بميدان دراسته)\nرابعا: كم من شخص وضع التخصص الذي يريده أولا ثم أهمل باقي الرغبات و لسوء حظه كان من نصيبه إحدى التخصصات التي لم يلقي لها بالا و وضعها في الترتيب الثاني أو الثالث أو حتى الرابع، و لهذا احسنوا الترتيب، فالتخصصات التي ستجدونها ليست كلها تدرس بمعدلكم."}
        ]
    },
  ##########################################
  {
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "كيف يمكنني اكتشاف اهتماماتي الأكاديمية واختيار تخصص مناسب؟"},
        {"role": "assistant", "content": "أولا: قم بتجربة مواد دراسية متنوعة لاستكشاف ميولك واهتماماتك. ثانيا: شارك في أنشطة طلابية وورش عمل لتوسيع أفقك الأكاديمي. ثالثا: استفد من استشارات المرشدين الأكاديميين للحصول على توجيهات حول اختيار التخصص المناسب لك."}
    ]
},
  ##########################################
  {
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "ما هي أهم النصائح لاختيار التخصص الجامعي؟"},
        {"role": "assistant", "content": "أولا: اكتشف ميولك واهتماماتك، حيث يساعد ذلك في اختيار تخصص يناسبك. ثانيا: استشر أساتذة ومتخصصين في المجال للحصول على آراءهم وتوجيهاتهم. ثالثا: قارن بين التخصصات المختلفة واستفد من تجارب الطلاب السابقين في هذه التخصصات."}
    ]
},
  ##########################################
  {
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "كيف يمكنني اتخاذ قرار مستنير في اختيار تخصص الدراسة مع النظر إلى مستقبلي المهني؟"},
        {"role": "assistant", "content": "أولا: استكشف التخصصات التي تتيح لك فرصًا واعدة في سوق العمل. ثانيا: قم بتحليل اتجاهات السوق واحتياجات الصناعة المستقبلية. ثالثا: اتخذ قرارك بناءً على توقعات مستقبلية واحتمالات النجاح في المجال الذي تختاره."}
    ]
},
##########################################
   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ما هي أهم التخصصات الجامعية"},
            {"role": "assistant", "content": "كما هو معروف التخصصات الجامعية كثيرة وكل تخصص لديه مايميزه عن باقي التخصصات لذا فان هذا السؤال قد يكون خطأ يجب أن تقول ما هي أهم التخصصات الجامعية التي تناسبني بمعنى توافق رغباتي و طموحاتي و شخصيتي"}
        ]
    },

########################################################################################################################################################################################################################################

 {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "الأوائل في بكالوريا 2023"},
            {"role": "assistant", "content": "المرتبة 1  بن قداش محمد الأمين الذي حصل على  معدل 19.5\nالمرتبة 2  دعاشي أسامة بمعدل 19.32\nالمرتبة 3 هلال ايمان فاطمة الزهراء بمعدل 19.32"}
        ]
    },
   ##########################################

   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "من هم اوائل باكالوريا 2023"},
            {"role": "assistant", "content": "المرتبة 1  بن قداش محمد الأمين الذي حصل على  معدل 19.5\nالمرتبة 2  دعاشي أسامة بمعدل 19.32\nالمرتبة 3 هلال ايمان فاطمة الزهراء بمعدل 19.32"}
        ]
    },
  ##########################################
   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "اذكر اوائل بكالوريا 2023"},
            {"role": "assistant", "content": "المرتبة 1  بن قداش محمد الأمين الذي حصل على  معدل 19.5\nالمرتبة 2  دعاشي أسامة بمعدل 19.32\nالمرتبة 3 هلال ايمان فاطمة الزهراء بمعدل 19.32"}
        ]
    },

   ########################################################################################################################################################################################################################################
 {"messages": [{"role": "system", "content": system_message},{"role": "user", "content": " هل يمكنني أن أتغيب عن الدروس في في الثانوية ؟"},{"role": "assistant", "content": "من المفضل المواظبة على الذهاب الى المدرسة يوميا و عدم التغيب في أية حصة بعض التلاميذ يتغيبون و يعتبرون الذهاب الى المدرسة مضيعة للوقت و لكن هذا خطا فادح لان التلميد يتوجب عليه أن يداوم على حصص المراجعة مع أساتذته الذين يمتلكون الخبرة و بمقدورهم إعطائه النصائح القيمة حتى أن هناك بعض التعديلات التي يمكن أن تغيب عن باله و يضيع فيها نقاطا سدى"}]},
 {"messages": [{"role": "system", "content": system_message},{"role": "user", "content": " هل يمكنني أن أتغيب عن الدروس في في الثانوية في الفصل الثاتي ؟"},{"role": "assistant", "content": "من المفضل المواظبة على الذهاب الى المدرسة يوميا و عدم التغيب في أية حصة بعض التلاميذ يتغيبون و يعتبرون الذهاب الى المدرسة مضيعة للوقت و لكن هذا خطا فادح لان التلميد يتوجب عليه أن يداوم على حصص المراجعة مع أساتذته الذين يمتلكون الخبرة و بمقدورهم إعطائه النصائح القيمة حتى أن هناك بعض التعديلات التي يمكن أن تغيب عن باله و يضيع فيها نقاطا سدى"}]},
 {"messages": [{"role": "system", "content": system_message},{"role": "user", "content": " هل يمكنني أن أتغيب عن الدروس في في الثانوية اذا لم اجد فائدة ؟"},{"role": "assistant", "content": "من المفضل المواظبة على الذهاب الى المدرسة يوميا و عدم التغيب في أية حصة بعض التلاميذ يتغيبون و يعتبرون الذهاب الى المدرسة مضيعة للوقت و لكن هذا خطا فادح لان التلميد يتوجب عليه أن يداوم على حصص المراجعة مع أساتذته الذين يمتلكون الخبرة و بمقدورهم إعطائه النصائح القيمة حتى أن هناك بعض التعديلات التي يمكن أن تغيب عن باله و يضيع فيها نقاطا سدى"}]},
 {"messages": [{"role": "system", "content": system_message},{"role": "user", "content": " هل يمكنني أن أتغيب عن الدروس في في الثانوية اذا كنت افضل الدراسة مفردا ؟"},{"role": "assistant", "content": "من المفضل المواظبة على الذهاب الى المدرسة يوميا و عدم التغيب في أية حصة بعض التلاميذ يتغيبون و يعتبرون الذهاب الى المدرسة مضيعة للوقت و لكن هذا خطا فادح لان التلميد يتوجب عليه أن يداوم على حصص المراجعة مع أساتذته الذين يمتلكون الخبرة و بمقدورهم إعطائه النصائح القيمة حتى أن هناك بعض التعديلات التي يمكن أن تغيب عن باله و يضيع فيها نقاطا سدى"}]},

  ########################################################################################################################################################################################################################################

  {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "كيفية اختيار موضوع البكالوريا"},
            {"role": "assistant", "content": "- أكثر موضوع سهل في الاختيار غالبا ما يكون في متناول جميع التلاميذ لأنه سهل ومفهوم في الغالب غير معقد.\n- تقوم بقراءة الموضوعين قراءة أولية ما يسمح لك من خلالها التعرف مثلا هل نوع الدالة الأسية، أو لوغاريتمية. يعني نظرة أولية عامة على طبيعة الموضوعين.\n- بعد القراءة الثانية ستكون لتحديد ما تعرف ومتيقن من الإجابة عنه وما لا تعرفه بمعنى آخر ضع علامة + أمام جميع الأسئلة التي تعرفها وعلامة * في الحالة العكسية.\n- الآن حاول معرفة العلامة التقريبية التي من المحتمل أن تحصل عليها من كلا الموضوعين، لكن لا تقوم بحساب عدد الأسئلة لأنه من الممكن أن عدد الأسئلة التي لم تعرفها عليها نقاط كثيرة، وإنما احسب عدد النقاط التي من المحتمل أن تحصل عليها من عدد الأسئلة التي تكون في المتناول.\n- بعد اختيار الموضوع ، ضع الموضوع الآخر جانبا ولا تلتفت له مرة أخرى مهما كان، افصل الورقة الخاصة به وضعها بعيدا ) وركز جيدا مع الموضوع الذي وقع عليه الاختيار ).\n- توكل على الله وابدأ في في حل الموضوع الذي تم اختياره وحاول الاجابة على كل الأسئلة ولا تقم بتغيير الموضوع ان وجدت سؤال صعب حاول حله أو اتركه وحاول معه في وقت لاحق بعد الانتهاء من جميع الاسئلة لكي لا يضيع وقتك.\n- عموما الوقت اللازم المستغرق في اختيار الموضوع بالنسبة لمادة الرياضيات يكون من 15 دقيقة الى 25 دقيقة."}
        ]
    },
 ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "كيف اختار موضوع البكالوريا"},
            {"role": "assistant", "content": "- أكثر موضوع سهل في الاختيار غالبا ما يكون في متناول جميع التلاميذ لأنه سهل ومفهوم في الغالب غير معقد.\n- تقوم بقراءة الموضوعين قراءة أولية ما يسمح لك من خلالها التعرف مثلا هل نوع الدالة الأسية، أو لوغاريتمية. يعني نظرة أولية عامة على طبيعة الموضوعين.\n- بعد القراءة الثانية ستكون لتحديد ما تعرف ومتيقن من الإجابة عنه وما لا تعرفه بمعنى آخر ضع علامة + أمام جميع الأسئلة التي تعرفها وعلامة * في الحالة العكسية.\n- الآن حاول معرفة العلامة التقريبية التي من المحتمل أن تحصل عليها من كلا الموضوعين، لكن لا تقوم بحساب عدد الأسئلة لأنه من الممكن أن عدد الأسئلة التي لم تعرفها عليها نقاط كثيرة، وإنما احسب عدد النقاط التي من المحتمل أن تحصل عليها من عدد الأسئلة التي تكون في المتناول.\n- بعد اختيار الموضوع ، ضع الموضوع الآخر جانبا ولا تلتفت له مرة أخرى مهما كان، افصل الورقة الخاصة به وضعها بعيدا ) وركز جيدا مع الموضوع الذي وقع عليه الاختيار ).\n- توكل على الله وابدأ في في حل الموضوع الذي تم اختياره وحاول الاجابة على كل الأسئلة ولا تقم بتغيير الموضوع ان وجدت سؤال صعب حاول حله أو اتركه وحاول معه في وقت لاحق بعد الانتهاء من جميع الاسئلة لكي لا يضيع وقتك.\n- عموما الوقت اللازم المستغرق في اختيار الموضوع بالنسبة لمادة الرياضيات يكون من 15 دقيقة الى 25 دقيقة."}
        ]
    },
 ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "على اي اساس اختار موضوع البكالوريا "},
            {"role": "assistant", "content": "- أكثر موضوع سهل في الاختيار غالبا ما يكون في متناول جميع التلاميذ لأنه سهل ومفهوم في الغالب غير معقد.\n- تقوم بقراءة الموضوعين قراءة أولية ما يسمح لك من خلالها التعرف مثلا هل نوع الدالة الأسية، أو لوغاريتمية. يعني نظرة أولية عامة على طبيعة الموضوعين.\n- بعد القراءة الثانية ستكون لتحديد ما تعرف ومتيقن من الإجابة عنه وما لا تعرفه بمعنى آخر ضع علامة + أمام جميع الأسئلة التي تعرفها وعلامة * في الحالة العكسية.\n- الآن حاول معرفة العلامة التقريبية التي من المحتمل أن تحصل عليها من كلا الموضوعين، لكن لا تقوم بحساب عدد الأسئلة لأنه من الممكن أن عدد الأسئلة التي لم تعرفها عليها نقاط كثيرة، وإنما احسب عدد النقاط التي من المحتمل أن تحصل عليها من عدد الأسئلة التي تكون في المتناول.\n- بعد اختيار الموضوع ، ضع الموضوع الآخر جانبا ولا تلتفت له مرة أخرى مهما كان، افصل الورقة الخاصة به وضعها بعيدا ) وركز جيدا مع الموضوع الذي وقع عليه الاختيار ).\n- توكل على الله وابدأ في في حل الموضوع الذي تم اختياره وحاول الاجابة على كل الأسئلة ولا تقم بتغيير الموضوع ان وجدت سؤال صعب حاول حله أو اتركه وحاول معه في وقت لاحق بعد الانتهاء من جميع الاسئلة لكي لا يضيع وقتك.\n- عموما الوقت اللازم المستغرق في اختيار الموضوع بالنسبة لمادة الرياضيات يكون من 15 دقيقة الى 25 دقيقة."}
        ]
    },
  ##########################################
   {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ما رأيك في الدروس الخصوصية و هل تنصحني أن ألتحق بها ؟"},
            {"role": "assistant", "content": "هذا يعتمد على مستوى أداء الأساتذة في القسم وعلى الطالب نفسه وقناعاته الشخصية.... الدروس الخصوصية ليست هي \"الأساس\" ولكنها مهمة خاصة للتلاميذ المتوسطين وحتى الممتازين للتمكن من استدراك بعض الجزئيات والتدرب على التمارين والالتقاء بالزملاء. ولكن مؤخرا انتشر اعتقاد راسخ لدى التلاميذ وأوليائهم أن الدروس الخصوصية هي كل شيء ما أدى الى تغيب التلاميذ عن دروسهم في القسم وهذا أمر غير مقبول تماما ففي الأخير نجاح التلميذ متوقف على تركيزه في القسم ومدى اجتهاده الشخصي وبحثه الدؤوب عن التمارين والأفكار الجديدة ومدى طرحه للأسئلة وسعيه لإيجاد الإجابات...أما إن كان التلميذ يتلقى التعليم الكافي في القسم مع اساتذة محنكين فلا داعي للدروس الخصوصية ـ انا عن نفسي ندمت لاني تلقيت البعض منها لانها كانت مضيعة فعلية للوقت والمال ـ"}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ما رأيك في الدروس الخصوصية و هل تنصحني أن ألتحق بها ؟"},
            {"role": "assistant", "content": " أرى أن الدروس الخصوصية لا تشكل \"الأساس\" بل تظل أمرًا ضروريًا للتلاميذ المتوسطين وحتى المتقدمين، حيث تمكِّنهم من تفادي بعض النقاط الصعبة وتطوير مهاراتهم من خلال ممارسة التمارين والتفاعل مع زملائهم. ولكن، ظهر مؤخرًا اعتقادٌ راسخٌ بين التلاميذ وأوليائهم يفيد بأن الدروس الفردية هي المفتاح الوحيد للنجاح، مما أدى إلى غياب الطلاب عن الحصص الدراسة في القسم، وهو أمر غير مقبول تمامًا. في الختام، يعتمد نجاح الطالب على تركيزه في الفصل واجتهاده الشخصي، والبحث الدؤوب عن التمارين والأفكار الجديدة."}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ما رأيك في الدروس الخصوصية و هل تنصحني أن ألتحق بها ؟"},
            {"role": "assistant", "content": "من وجهة نظري، الدروس الخصوصية لا تمثل الأساس بل تعد أمرًا ضروريًا للتلاميذ الذين يتوسطون مستوى الأداء الدراسي، وحتى للمتميزين، لأنها تمكنهم من تصحيح بعض الفهم الصعب وتنمية مهاراتهم من خلال التدرب على التمارين والتفاعل مع زملائهم. ولكن، طرأ اعتقاد جذري بين الطلاب وأوليائهم يرى أن الدروس الخصوصية هي كل شيء، مما أدى إلى تغيب الطلاب عن حضور الدروس الجماعية في الصف، وهو أمر غير مقبول تمامًا. في الختام، يكمن نجاح الطالب في التركيز داخل الفصل واجتهاده الشخصي، واستمراره في البحث عن التمارين والأفكار الجديدة، بالإضافة إلى طرح الأسئلة والسعي للعثور على الإجابات. إذا كان الطالب يستفيد من تعليم كاف في الصف مع معلمين ماهرين، فلا داعي للدروس الخصوصية.ففي هده الحالة هي تمثل إضاعة فعلية للوقت والمال."}
        ]
    },
  ##########################################
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "ما رأيك في الدروس الخصوصية و هل تنصحني أن ألتحق بها ؟ "},
            {"role": "assistant", "content": "الدروس الخصوصية مفيدة جدا أنصح كل الطلبة بها لكن شرط أن لا يكون عدد التلاميذ كبيرا وإلا فلا فائدة منها"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "كيف أتعامل مع المواد العلمية ؟"},
            {"role": "assistant", "content": " من وجهة نظري، لا أعتبر الدروس الخصوصية \"الأساس\"، ولكنها قد تكون مفيدة للتلاميذ في المستويات المتوسطة وحتى الطلاب المتميزين. تساعد هذه الدروس في تعزيز فهمهم لبعض النقاط الصعبة، وتوفر لهم الفرصة للتدريب على التمارين والتفاعل مع زملائهم. ومع ذلك، انتشر مؤخراً اعتقاد راسخ بين الطلاب وأوليائهم يرى أن الدروس الخصوصية هي كل شيء، مما أدى إلى تغيب الطلاب عن الحصص الجماعية في الصف، وهو أمر غير مقبول تماماً. في الختام، يعتمد نجاح الطالب على تركيزه داخل الفصل واجتهاده الشخصي، واستمراره في البحث عن التمارين والأفكار الجديدة، بالإضافة إلى طرح الأسئلة والسعي للعثور على الإجابات. إذا كان الطالب يستفيد من تعليم كاف في الصف مع معلمين ماهرين، فلا داعي للدروس الخصوصية.ففي هده الحالة هي تمثل إضاعة فعلية للوقت والمال."}
        ]
    }

  ########################################################################################################################################################################################################################################





]

In [ ]:
len(dataset)

69

\\### converting the conversation to correct format

In [ ]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 140, 929
mean / median: 278.71014492753625, 203.0
p5 / p95: 162.4, 635.2

#### Distribution of num_assistant_tokens_per_example:
min / max: 8, 789
mean / median: 133.82608695652175, 55.0
p5 / p95: 24.0, 488.8

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 30
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25


n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")


Dataset has ~19231 tokens that will be charged for during training
By default, you'll train for 30 epochs on this dataset
By default, you'll be charged for ~576930 tokens
See pricing page to estimate total costs


In [ ]:
import json

def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [ ]:
# train dataset
save_to_jsonl(dataset, '/content/Bac-gpt_taskstrain.jsonl')

# train dataset
save_to_jsonl(dataset[45:], '/content/Bac-gpt_tasks_validation.jsonl')

## Upload your data

In [ ]:
# curl -https://api.openai.com/v1/files \
#   -H "Authorization: Bearer $OPENAI_API_KEY" \
#   -F "purpose=fine-tune" \
#   -F "file=@path_to_your_file"

In [ ]:
training_file_name = '/content/Bac-gpt_taskstrain.jsonl'
validation_file_name = '/content/Bac-gpt_tasks_validation.jsonl'

In [ ]:


training_response = client.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)


training_file_id = training_response.id


validation_response = client.files.create(
  file=open(validation_file_name , "rb"),
  purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file id:", training_file_id)


Training file id: file-V2GHBvCxFwivKlrW2MWIHTtz


## Create a Fine Tuning Job

In [ ]:
# curl https://api.openai.com/v1/fine_tuning/jobs \
# -H "Content-Type: application/json" \
# -H "Authorization: Bearer $OPENAI_API_KEY" \
# -d '{
#   "training_file": "TRAINING_FILE_ID",
#   "model": "gpt-3.5-turbo-0613",
# }'

In [ ]:
suffix_name = "Bac-GPT"

response = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  validation_file=validation_file_id,
  model="gpt-3.5-turbo-1106",
  suffix=suffix_name,
  hyperparameters={
    "n_epochs": 2

  }
)




print(response)

FineTuningJob(id='ftjob-X7EiBxJEOo9XVYUZx7x5hzEC', created_at=1702079368, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-ChtvB8ZoouVV6zzwOOLkSTkV', result_files=[], status='validating_files', trained_tokens=None, training_file='file-V2GHBvCxFwivKlrW2MWIHTtz', validation_file='file-W1wyknarn1ura1lrx4NqyOxl')


In [ ]:
job_id = response.id

In [ ]:
# response = openai.FineTuningJob.list_events(id=job_id, limit=50)
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit= 10)


events = response.data
events.reverse()



for event in events:
    print(event.message)


Step 61/138: training loss=2.15, validation loss=0.34
Step 71/138: training loss=0.39, validation loss=0.56
Step 81/138: training loss=1.01, validation loss=0.88
Step 91/138: training loss=1.03, validation loss=0.82
Step 101/138: training loss=0.79, validation loss=0.79
Step 111/138: training loss=0.36, validation loss=0.78
Step 121/138: training loss=1.40, validation loss=0.83
Step 131/138: training loss=0.91, validation loss=0.82
New fine-tuned model created: ft:gpt-3.5-turbo-1106:personal:bac-gpt:8TeyxFLg
The job has successfully completed


In [ ]:
response = client.fine_tuning.jobs.retrieve(job_id)

fine_tuned_model_id = response.fine_tuned_model

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

FineTuningJob(id='ftjob-X7EiBxJEOo9XVYUZx7x5hzEC', created_at=1702079368, error=None, fine_tuned_model='ft:gpt-3.5-turbo-1106:personal:bac-gpt:8TeyxFLg', finished_at=1702079738, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-ChtvB8ZoouVV6zzwOOLkSTkV', result_files=['file-zQj1Ery5PHzC51EtTmODKdNM'], status='succeeded', trained_tokens=38186, training_file='file-V2GHBvCxFwivKlrW2MWIHTtz', validation_file='file-W1wyknarn1ura1lrx4NqyOxl')

Fine-tuned model id: ft:gpt-3.5-turbo-1106:personal:bac-gpt:8TeyxFLg


## Generating using the new model

In [ ]:
system = """
انت مساعد شخصي لتلاميذ باكالوريا الجزائر
عليك ان تجيب عللا اسئلتهم في حدود اليانات التي تم تدريبك عليها
حاول ان تجعل الرسالة طويلة
"""


test_messages = []
user_message = "اذكر أهم الكتب الخارجية لمادة العربية / الادب العربي / اللغة العربية"
test_messages.append({"role": "system", "content": system})
test_messages.append({"role": "user", "content": user_message})

response = client.chat.completions.create(
    model=fine_tuned_model_id,
    messages=test_messages,
    temperature=0.15,
    max_tokens=500
)





#اذكر أهم الكتب الخارجية لمادة التاريخ و الجغرافيا
#اذكر اوائل بكالوريا 2023
#ما هي أهم التخصصات الجامعية"
#اذكر أهم الكتب الخارجية للغات الأجنبية (الإنجليزية الفرنسية الألمانية الاسبانية )"

response.choices[0].message.content

'كتاب السلسلة الأرجوانية \nكتاب السلسلة الخضراء \nكتاب السلسلة الحمراء'

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

def generate_response(user_message):
    # Add your existing code here
    system_message = """
    You should only answer from the data you have been trained on
    you should only reformulate it here is and example:
    user: شكون استاذ مليح تاريخ وجغرافيا"
    answer reformulated : احسن اساتذة هم : الاستاذ عبدوش مختار و الاستاذ عمار بورنان
    """
    test_messages = [{"role": "system", "content": system_message},
                     {"role": "user", "content": user_message}]

    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=test_messages,
        temperature=0.6,
        max_tokens=500
    )

    return

@app.route('/generate_response', methods=['POST'])
def post():
    data = request.get_json()
    user_message = data['text']
    response_text = generate_response(user_message)

    return jsonify({'response': response_text})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
